# Knowdge Distillation using Contrastive Learning


 #### Student Name:


 #### Student ID:


In this exercise, we aim to distill knowledge from a large monolingual model into a smaller multilingual model using contrastive learning, specifically leveraging the CLIP model loss.

We employ a small paired English-Persian dataset to define the loss pairs for our CLIP training. Given the substantial dataset size and batch size typically required for CLIP's loss computation (exceeding 19,000 samples per batch in standard tasks), which is impractical for our setup on Colab, we use a reduced batch size to focus on learning the procedure rather than achieving optimal performance, so we don't expect actual real-world results, only the training prcodure.

CLIP (Contrastive Language-Image Pretraining) is a foundational model introduced by OpenAI to bridge the gap between text and image modalities. By aligning text descriptions and corresponding images in a shared embedding space, CLIP achieves remarkable zero-shot generalization capabilities across a wide range of tasks. It is trained on a massive dataset of image-text pairs using contrastive loss, ensuring that image embeddings align closely with their corresponding textual descriptions while remaining distinct from unrelated samples. This cross-modal alignment enables CLIP to perform tasks like image retrieval, captioning, and classification with minimal fine-tuning.

The CLIP loss plays a crucial role in training the model by implementing a cross-entropy loss function in the contrastive learning framework. This loss operates on paired data, where each image-text pair is treated as a positive match, while all other pair combinations in the batch are considered negatives. The loss ensures that positive pairs receive high similarity scores, while negatives are penalized. However, achieving optimal results with CLIP loss often requires large batch sizes to provide sufficient negative samples, which can be computationally intensive. This makes training with limited resources challenging, necessitating adaptations such as smaller batch sizes or alternative strategies to approximate the training dynamics.


Knowledge distillation is a technique used in machine learning to transfer knowledge from a large, complex model (the "teacher") to a smaller, more efficient model (the "student"). The primary goal of this process is to retain the performance and accuracy of the larger model while significantly reducing computational and memory requirements. This is achieved by training the student model to mimic the outputs of the teacher model, often through techniques such as matching soft probability distributions or intermediate representations. Knowledge distillation has become an essential approach in deploying machine learning models on resource-constrained devices such as smartphones and edge devices.

In practice, knowledge distillation is not limited to replicating predictions; it can also involve transferring knowledge about internal features or learned representations.


### Challenges in Resource-Constrained Settings
CLIP’s reliance on large-scale datasets and batch sizes makes direct implementation computationally demanding. This exercise demonstrates an adaptation of the process, reducing batch size and dataset size to provide a practical understanding of the training procedure. While this approach sacrifices performance and real-world applicability, it highlights the mechanics of using CLIP loss for contrastive learning and lays the foundation for extending the process to larger datasets and batch sizes in future applications.



### About CLIP and Contrastive Learning
CLIP, developed by OpenAI, bridges the gap between text and image modalities by aligning corresponding embeddings in a shared space. It leverages contrastive loss to train on image-text pairs, ensuring that embeddings of positive pairs (e.g., an image and its corresponding caption) are highly similar, while embeddings of unrelated pairs remain distinct. The cross-entropy-based contrastive loss evaluates the similarity between positive pairs while penalizing mismatches for all other combinations within a batch.

### Key aspects of CLIP loss include:

- Positive Pairing: Encourages high similarity scores for embeddings of paired text and image data.
- Negative Sampling: Penalizes mismatched pairs within the batch, requiring large batch sizes for effective performance due to the need for a diverse set of negative samples.



## !!! Note !!! You Should Answer to all the TODOs

Also feel free to ask your questions on Quera.



## Setup


We install Required Packages.

In [ ]:
!pip install -q gdown
!gdown "https://drive.google.com/uc?id=1MVx_gIkX4tQ8ya2OsHt0mqLmw1Pf2CcK"
!gdown "https://drive.google.com/uc?id=1Co-dwJfWw-C_ral0hoAS_X94wN-_vbCj"

In [ ]:
!pip install setuptools
import sys
import subprocess
import pkg_resources

def installPackages(packages):
    def installPackage(package):
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

    for package in REQUIRED_PACKAGES:
        try:
            dist = pkg_resources.get_distribution(package)
            print('{} ({}) is installed'.format(dist.key, dist.version))
        except pkg_resources.DistributionNotFound:
            print('{} is NOT installed'.format(package))
            installPackage(package)
            print('{} was successfully installed.'.format(package))

REQUIRED_PACKAGES = [
    'open_clip-torch',
    'pandas',
    'numpy',
    'matplotlib',
    'transformers',
    'tqdm',
    'torch',
    'datasets',
]

installPackages(REQUIRED_PACKAGES)

import gc
import itertools
import re
import math
from collections import Counter
import random
import string
import uuid

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import open_clip
from open_clip import model as TE

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

from datasets import load_dataset, Dataset, Features, Array2D, Value
from transformers import AutoConfig, AutoTokenizer, AutoModel, TFAutoModel

from tqdm import tqdm

Beware to use cuda for training!

In [ ]:
def getDevice(which="cuda:0", yellAtCpu=True):
    if torch.cuda.is_available():
        device = torch.device(which)
    else:
        if yellAtCpu:
             raise Exception("I won't run on CPU!")
        device = torch.device("cpu")

    return device

print(f"Device: {getDevice()}")


#### Configs

These are our training configurations, read them!

In [ ]:
def Configs():
    return {
        "device": getDevice(),
        "reference_checkPoint" : "EVA02-E-14-plus",                # teacher
        "candidate_checkpoint" : "setu4993/smaller-LaBSE",         # student
        "train_path" : "train.csv",
        "val_path" : "val.csv",
        "save_path" : "./best-model.pth",
        "english" : "en",                                         # dont mind them
        "persian" : "fa",
        "batch_size": 128,                                        # should have been really big, but we can't here
        "lr": 1e-4,
        "epochs": 5,                                               # 40 minute per epoch
        "tok_percentile" : 99,
        "temperature": 20,
        "dropout": 0.05,
        "unfreezed_layers" : 10,
        "weight_decay": 1e-5,
        "patience": 1,
        "factor" : 0.8,
        "reference_embedding": 1024,                               # DONT MIND THESE
        "reference_context_length" : 77,
        "reference_vocab_size" : 49408,
        "reference_heads" : 20,
        "reference_width" : 1280,
        "reference_layers" : 32,
        "cls_token_index" : 0,
        "project_to" : 1024,
    }

configs = Configs()

## Question Box
### TODO (10pts)
1- Why do we use temperature in training using contrastive learning?<br/><br/>

  In contrastive learning, temperature is a hyperparameter used to scale the logits before they are passed to the softmax function. The purpose of the temperature parameter is to control the softness of the probability distribution produced by the softmax function, so that it could help with better separation of positive pairs from negative pairs. Higher temperature leads to a softer distribution, and lower temperature leads to a harder distribution. Temperature is crucial to tune the hardness or softness of the distribution in similarity space.
<br/><br/>
2- Why do we need to freeze some layers of a model? mention 2 reasons.<br/><br/>
  Freezing layers means making their weights untrainable during backpropagation. This is done for several reasons:

    Reason 1: Preserve Pre-trained Knowledge: Pre-trained models often come with valuable knowledge about general features. Freezing initial layers prevents their weights from being drastically altered, preserving this knowledge and allowing the model to focus on learning task-specific features in the later layers.
    Reason 2: Reduce Computational Cost: Freezing layers reduces the number of parameters being updated during training, leading to faster training and lower memory requirements. This is particularly important when fine-tuning large pre-trained models on limited resources.


<br/>
3- Read the whole code and find out what tok_percentile is used for.<br/><br/>

tok_percentile is used to determine the maximum length of input sequences after tokenization. It is set to 99, meaning that 99% of the sentences in the training set are at or below the chosen token length, and only 1% that might be longer will be truncated. This helps ensure that the vast majority of sentences in the training dataset are included during training without exceeding a defined token length. This helps in preventing very long input texts and maintain uniform input lengths for the training.

## Data and Preprocessing

In [ ]:
def getDatasetsCSV(prevEnCol, prevFaCol, newEnCol, newFaCol, trainPath, valPath):
    df = pd.read_csv(trainPath)
    dfVal = pd.read_csv(valPath)

    if df.empty:
        raise ValueError("Training dataset is empty or missing")

    if dfVal.empty:
        raise ValueError("Validation dataset is empty or missing")

    dfTraind = df.loc[:, [prevEnCol, prevFaCol]].rename(columns={prevEnCol: newEnCol, prevFaCol: newFaCol})
    dfVal = dfVal.loc[:, [prevEnCol, prevFaCol]].rename(columns={prevEnCol: newEnCol, prevFaCol: newFaCol})

    datasetTrain = Dataset.from_pandas(dfTraind)
    datasetVal = Dataset.from_pandas(dfVal)

    return datasetTrain, datasetVal

def getDsByLang(persianCol, englishCol):
    def getPersianDs(dataset):
        return dataset[persianCol]

    def getEnglishDs(dataset):
        return dataset[englishCol]

    getPersianDs.label = persianCol
    getEnglishDs.label = englishCol

    return getPersianDs, getEnglishDs

class Normalizer():
    def __init__(self):
        translation_src = ' ىكي“”0123456789%إأآئيؤةك'
        translation_dst = ' یکی""۰۱۲۳۴۵۶۷۸۹٪اااییوهک'

        self.translations = str.maketrans(translation_src, translation_dst)

        patterns = [
            (r' {2,}', ' '),  # remove extra spaces
            (r'\n+', ' '),  # replace newlines with space
            (r'\u200c+', ' '),  # replace ZWNJs with space
            (r'[ـ\r]', '')  # remove keshide, carriage returns
        ]

        self.character_refinement_patterns = [(re.compile(pattern), repl) for pattern, repl in patterns]

    def normalizeFa(self, text):
        text = text.lower().translate(self.translations)
        text = re.sub('[^a-zA-Z۰-۹آ-ی ]', ' ', text)

        for pattern, repl in self.character_refinement_patterns:
            text = pattern.sub(repl, text)
        return text.strip()

    def normalizeEn(self, text):
        text = text.lower()
        text = text.translate(str.maketrans('', '', string.punctuation)) # Remove punctuation
        return text

def applyPreprocess(datasets, configs=configs, Normalizer=Normalizer):
    def applyRowNormalization(example):
        example[configs['persian']] = normalizer.normalizeFa(example[configs['persian']])
        example[configs['english']] = normalizer.normalizeEn(example[configs['english']])

        return example

    normalizer = Normalizer()

    newDatasets = []
    for dataset in datasets:
        newDatasets.append(dataset.map(applyRowNormalization))

    return newDatasets

def preprocessSentence(text, lang, mostFreq=None, Normalizer=Normalizer, configs=configs):
    normalizer = Normalizer()
    if lang == configs['persian']:
        normalized = normalizer.normalizeFa(text)
    elif lang == configs['english']:
        normalized = normalizer.normalizeEn(text)
    else:
        raise ValueError("Not supported lang")

    return normalized

## Utils
### TODO: Complete these Utility functions (10pts)

In [ ]:
def getClsToken(tensor, configs=configs):
    """
    Extracts the classification (CLS) token from the input tensor.

    Parameters:
        tensor (torch.Tensor): The input tensor of shape (batch_size, seq_length, hidden_dim).
        configs (dict): A dictionary containing configuration settings. Must include the key "cls_token_index"
                        which specifies the index of the CLS token in the sequence dimension.

    Returns:
        torch.Tensor: A tensor containing the CLS token for each example in the batch,
                      of shape (batch_size, 1, hidden_dim).
    """
    clsId = configs["cls_token_index"]
    return tensor[:, clsId, :].unsqueeze(1)

def flattenMiddle(tensor):
    """
    Flattens the middle dimension (sequence length) of the input tensor, removing it.

    Parameters:
        tensor (torch.Tensor): The input tensor of shape (batch_size, seq_length, hidden_dim).

    Returns:
        torch.Tensor: A tensor of shape (batch_size, hidden_dim) with the middle dimension flattened out.
    """
    return tensor.squeeze(1)

def freezeModel(model):
    """
    Freeze all parameters of a given model.
    Parameters:
        model (torch.nn.Module): The PyTorch model whose parameters are to be frozen.
    """
    for param in model.parameters():
        param.requires_grad = False
    return model

In [ ]:
def plotMetric(metricData, metricName):
    if metricName == None or metricName not in metricData:
        raise ValueError("No such metric")
    metricData[metricName].plot()
    plt.xlabel('Epochs')
    plt.ylabel(metricName)
    plt.title(f'Plot of {metricName}')
    plt.show()

threshold=1
def calcPrcentileTokens(dataset, tokenizer, field, percentile=configs["tok_percentile"], thershold=1):
    """
    Calculate the token length at a specific percentile for a dataset field.

    This function tokenizes the data in the specified field of the dataset and calculates
    the token length at the given percentile. An optional threshold can be added to the result.

    Parameters:
        dataset (dict or Dataset): The dataset containing the data to be tokenized.
        tokenizer (callable): A tokenizer function or object with a callable interface
                              (e.g., HuggingFace tokenizer).
        field (str): The field in the dataset whose token lengths are to be calculated.
        percentile (float, optional): The percentile to compute (default is the value in
                                       `configs["tok_percentile"]`).
        thershold (int, optional): A value to add to the calculated percentile token length
                                    (default is 1).

    Returns:
        int: The token length at the specified percentile plus the threshold.

    Raises:
        KeyError: If the specified field does not exist in the dataset.
        TypeError: If `tokenized` is not in the expected format.

    Example:
        dataset = {"text": ["This is a sentence.", "Another example sentence."]}
        tokenizer = lambda x: {"input_ids": [[1, 2, 3, 4], [5, 6, 7, 8, 9]]}
        field = "text"
        calcPrcentileTokens(dataset, tokenizer, field, percentile=95, thershold=2)

    Notes:
        - If the tokenized output is a dictionary (e.g., HuggingFace tokenizers), it assumes
          that `input_ids` contains the token sequences.
        - If the tokenized output is a tensor, nonzero token counts are used to determine lengths.
    """
    tokenized = tokenizer(dataset[field])
    if not isinstance(tokenized, torch.Tensor):
        tokenLengths = list(map(lambda sen: len(sen), tokenized['input_ids']))
    else:
        tokenLengths = [tensor.nonzero().size(0) for tensor in tokenized]
    percentileLength = np.percentile(tokenLengths, percentile)
    return int(percentileLength) + 1

# Dont Touch This
def TextEncoder(configs):
    newModel = TE.TextTransformer(context_length=configs['reference_context_length'],
                                 vocab_size=configs["reference_vocab_size"],
                                 width=configs["reference_width"],
                                 layers=configs["reference_layers"],
                                 heads=configs["reference_heads"],
                                 output_dim=configs["reference_embedding"])
    return newModel


## Models

### TODO: Compelete the Swish and LinearProjection functions based on the pydoc provided (15pts)

In [ ]:
class Swish(nn.Module):
    """
    Implements the Swish activation function.
    Parameters:
        beta (float, optional): The scaling parameter for the input x in the sigmoid
                                function. Default is 1.0.
    """
    def __init__(self, beta=1.0):
        super().__init__()
        self.beta = beta

    def forward(self, x):
        return x * torch.sigmoid(self.beta * x)

class LinearProjection(nn.Module):
    """
    A projection layer with Swish activation, batch normalization, dropout, and residual connections.

    This module takes an input tensor, applies a series of transformations, and produces an
    output tensor of the same shape, making use of residual connections and layer normalization.

    Parameters:
        embedding_dim (int): The dimensionality of the input embeddings.
        projection_dim (int, optional): The dimensionality of the projection. Default is
                                        `configs['project_to']`.
        dropout (float, optional): The dropout rate. Default is `configs['dropout']`.

    Layers:
        - projection: Linear layer that projects the input to the specified `projection_dim`.
        - swish: Swish activation function with a fixed beta of 1.0.
        - batch_norm: Batch normalization applied after projection.
        - fc: Fully connected layer for further transformations.
        - dropout: Dropout applied to the output of the fully connected layer.
        - layer_norm: Layer normalization applied after residual connection.

    Methods:
        forward(x):
            Applies the projection, activation, normalization, dropout, residual connection,
            and layer normalization to the input tensor.

    Example:
        model = LinearProjection(embedding_dim=128, projection_dim=64, dropout=0.1)
        output = model(input_tensor)
    """
    def __init__(self, embedding_dim, projection_dim=configs['project_to'], dropout=configs['dropout']):
        super(LinearProjection, self).__init__()
        self.projection = nn.Linear(embedding_dim, projection_dim)
        self.swish = Swish(beta=1.0)
        self.batch_norm = nn.BatchNorm1d(projection_dim)
        self.fc = nn.Linear(projection_dim, projection_dim)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(projection_dim)

    def forward(self, x):
        projected = self.projection(x)
        x = self.swish(projected)
        x = self.batch_norm(x)
        x = self.fc(x)
        x = self.dropout(x)
        x = self.layer_norm(x + projected)  # Residual connection
        return x

In [ ]:
class CandidateModel(nn.Module):
    def __init__(self, model_name, unfreezeLayers, trainable=True):
        super().__init__()
        self.candidateProjection = LinearProjection(embedding_dim=configs["candidate_embedding"])
        self.configs = AutoConfig.from_pretrained(model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.batchNorm = nn.BatchNorm1d(1, configs["candidate_embedding"])
        self.targetTokenIdx = configs["cls_token_index"]

    def forward(self, input_ids, attention_mask):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask)
        clsEmbed = getClsToken(output.last_hidden_state)
        clsEmbed = self.batchNorm(clsEmbed)
        clsEmbed = self.candidateProjection(flattenMiddle(clsEmbed))
        return clsEmbed

## Training

### TODO: Compelete the calcLoss functions based on the pydoc provided (20pts)

In [ ]:
def calcLoss(batch, referenceModel, candidateModel, temperature):
    """
    Compute the loss and the number of correct predictions for a contrastive learning task.

    This function calculates a symmetric cross-entropy loss between the embeddings
    generated by a reference model and a candidate model. It also computes the
    number of correct predictions based on the alignment of the embeddings.

    Args:
        batch (dict): A batch of data containing:
            - "candidate" (torch.Tensor): Tokenized input for the candidate model.
            - "reference" (torch.Tensor): Tokenized input for the reference model.
        referenceModel (nn.Module): The model generating embeddings for the reference input.
        candidateModel (nn.Module): The model generating embeddings for the candidate input.
        temperature (float): A scaling factor to control the logits' sharpness during similarity calculation.

    Returns:
        tuple:
            - loss (torch.Tensor): The computed symmetric cross-entropy loss.
            - corrects (int): The number of correctly predicted alignments.

    Notes:
        - The embeddings are normalized to ensure their magnitudes do not impact similarity.
        - Logits represent scaled cosine similarity between reference and candidate embeddings.
        - The targets are identity matrices, assuming perfect alignment between reference and candidate inputs.

    Example Usage:
        batch = {
            "candidate": tokenized_candidate,
            "reference": tokenized_reference
        }
        loss, corrects = calcLoss(batch, reference_model, candidate_model, temperature=0.1)
    """
    # Move tokenized inputs to the specified device
    candidateTokenized = batch["candidate"].to(configs["device"])
    referenceTokenized = batch["reference"].to(configs["device"])

    # Generate embeddings from reference and candidate models
    referenceEmbeds = referenceModel(referenceTokenized)
    candidateEmbeds = candidateModel(
        input_ids=candidateTokenized["input_ids"],
        attention_mask=candidateTokenized["attention_mask"]
    )

    # Normalize embeddings to have unit length
    referenceEmbeds = F.normalize(referenceEmbeds, p=2, dim=-1)
    candidateEmbeds = F.normalize(candidateEmbeds, p=2, dim=-1)

    # Calculate logits (scaled cosine similarity)
    logits = torch.matmul(candidateEmbeds, referenceEmbeds.t()) / temperature

    # Define targets (identity matrix)
    batch_size = candidateEmbeds.shape[0]
    targets = torch.arange(batch_size, device=configs["device"])

    # Calculate cross-entropy loss
    loss_candidate = F.cross_entropy(logits, targets)
    loss_reference = F.cross_entropy(logits.t(), targets)
    loss = (loss_candidate + loss_reference) / 2

    # Calculate correct predictions
    corrects = (logits.argmax(dim=1) == targets).sum().item()

    return loss, corrects

### TODO: Fill the trainLoop and valLoop (5pts)

In [ ]:
def trainLoop(dataloader, models, referenceTokenizer, candidateTokenizer, optimizer, temperature):
    models['candidateModel'].train()

    totalLoss = 0.0
    totalCorrects = 0

    print("Training Starts!")
    for (index, pairs) in tqdm(enumerate(dataloader), total=len(dataloader)):

        candidteTokenized = candidateTokenizer(getPersianDs(pairs), padding='max_length', truncation=True, return_tensors="pt", max_length=configs["fa_tok_percentile"])
        referenceTextTokenized = referenceTokenizer(getEnglishDs(pairs))

        batch = {
            "candidate" : candidteTokenized,
            "reference" : referenceTextTokenized
        }

        loss, corrects = calcLoss(batch, models['referenceModel'], models['candidateModel'], temperature)
        totalCorrects += corrects
        totalLoss += loss.item() * len(pairs) # Multiply by batch size

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    avgLoss = totalLoss / len(dataloader.dataset)
    avgAccuracy = totalCorrects / len(dataloader.dataset)

    print("train loss", avgLoss)
    print("train accuracy", avgAccuracy)

    return avgLoss, avgAccuracy

def valLoop(dataloader, models, referenceTokenizer, candidateTokenizer, temperature):
    models['candidateModel'].eval()

    print("Testing Starts!")
    totalLoss = 0.0
    totalCorrects = 0

    with torch.no_grad():
        for (index, pairs) in tqdm(enumerate(dataloader), total=len(dataloader)):
            candidteTokenized = candidateTokenizer(getPersianDs(pairs), padding='max_length', truncation=True, return_tensors="pt", max_length=configs["fa_tok_percentile"])
            referenceTextTokenized = referenceTokenizer(getEnglishDs(pairs))

            batch = {
                "candidate" : candidteTokenized,
                "reference" : referenceTextTokenized
            }

            loss, corrects = calcLoss(batch, models['referenceModel'], models['candidateModel'], temperature)

            totalCorrects += corrects
            totalLoss += loss.item() * len(pairs) # Multiply by batch size

    avgLoss = totalLoss / len(dataloader.dataset)
    avgAccuracy = totalCorrects / len(dataloader.dataset)

    print("test loss", avgLoss)
    print("test accuracy", avgAccuracy)

    return avgLoss, avgAccuracy

In [ ]:
oldCols = ["en", "fa"]
datasetTrain, datasetVal = getDatasetsCSV(oldCols[0], oldCols[1], configs["english"], configs["persian"], configs["train_path"], configs["val_path"])

getPersianDs, getEnglishDs = getDsByLang(configs["persian"], configs["english"])
print("Before Preproccess: ", datasetTrain[0])
datasetTrain, datasetVal = applyPreprocess([datasetTrain, datasetVal], configs)
print("After Preproccess: ", datasetTrain[0])
trainDataloader = DataLoader(datasetTrain, batch_size=configs['batch_size'], shuffle=True)
valDataloader = DataLoader(datasetVal, batch_size=configs['batch_size'], shuffle=False)

#### Tokenizers

In [ ]:
referenceTokenizer = open_clip.get_tokenizer(configs["reference_checkPoint"])

candidateConfig = AutoConfig.from_pretrained(configs["candidate_checkpoint"])
candidateTokenizer = AutoTokenizer.from_pretrained(configs["candidate_checkpoint"])

configs = configs | {"candidate_embedding" : candidateConfig.hidden_size}

faTokenPercentile = calcPrcentileTokens(datasetTrain, candidateTokenizer, configs["persian"])
faTokenPercentile

enTokenPercentile = calcPrcentileTokens(datasetTrain, referenceTokenizer, configs["english"])
enTokenPercentile

configs = configs | {"en_tok_percentile" : enTokenPercentile}
configs = configs | {"fa_tok_percentile" : faTokenPercentile}

#### Models

In [ ]:
referenceModel = TextEncoder(configs).to(configs["device"])
candidateModel = CandidateModel(model_name=configs["candidate_checkpoint"], unfreezeLayers=configs["unfreezed_layers"]).to(configs["device"])
referenceModel = freezeModel(referenceModel)

candidateModel.to(configs['device'])
referenceModel.to(configs['device'])

models = {
    "referenceModel" : referenceModel,
    "candidateModel" : candidateModel
}

## Training
### TODO (10pts) for running the code and (20pts) for achieving above 70 percent test accuracy

In [ ]:
temperature = torch.nn.Parameter(torch.tensor(configs['temperature']).float())
optimizer = torch.optim.AdamW(list(models['candidateModel'].parameters()) + [temperature], weight_decay=configs["weight_decay"], lr=configs['lr'])
lrScheduler = ReduceLROnPlateau(optimizer, 'max', patience=configs['patience'], factor=configs['factor'])


In [ ]:

bestValAcc = float('-inf')

metrics = pd.DataFrame(columns=["Avg-train-loss", "Avg-train-accuracy","Avg-val-loss", "Avg-val-accuracy"])

for t in range(configs['epochs']):
    trainLoss, trainAcc = trainLoop(trainDataloader, models, referenceTokenizer, candidateTokenizer, optimizer, temperature)
    valLoss, valAcc = valLoop(valDataloader, models, referenceTokenizer, candidateTokenizer, temperature)

    metrics.loc[t+1] = [trainLoss, trainAcc, valLoss, valAcc]

    lrScheduler.step(valAcc)

    print("Temperature at this epoch was :", temperature.item())

print(f'Best accuracy of validation gained: ', bestValAcc)
print("Done!")

Reached Colab limit for GPU usage :))))

In [ ]:
plotMetric(metrics, "Avg-val-accuracy")
metrics.tail(3)

## Question Box (10pts + 5pts bonus)

1- Do we even need normalization for tasks like this? will they provide any accuracy gain? write what you think.
<br/><br/>

Yes, normalization is generally beneficial in tasks like this, especially when using cosine similarity for contrastive learning. It can contribute to improved accuracy and training stability.

Reasoning:

    Improved Accuracy: Normalization ensures that the magnitude of the embeddings doesn't dominate the similarity calculation. By focusing on the direction of the embeddings, normalization allows the model to learn more meaningful relationships between the text and image representations. This often leads to better alignment and improved accuracy in contrastive learning tasks.

    Training Stability: Normalization can help stabilize the training process by preventing gradients from exploding or vanishing due to large variations in embedding magnitudes. This can lead to faster and more reliable convergence during training.

    Cosine Similarity: Cosine similarity is sensitive to the magnitude of vectors. If the vectors are not normalized, the similarity score will be biased towards vectors with larger magnitudes. Normalization removes this bias, allowing the model to focus on the direction of the vectors, which is more relevant to the semantic similarity of the texts.
<br/>
2- When training a neural network, what takes the memory? mention at least 4 things.
<br/><br/>


  Model Parameters: The weights and biases of the neural network are stored in memory. Larger models with more layers and parameters require more memory.<br/><br/>
  Activations: The intermediate outputs of each layer (activations) need to be stored during the forward pass to be used in the backward pass for gradient calculations. These activations can consume a significant amount of memory, especially for deep networks and large batch sizes.<br/><br/>
  Optimizer State: Optimizers like Adam or SGD store internal state variables (e.g., momentum, gradients) to update model parameters. This state also contributes to memory usage.<br/>
  Data: The training data itself needs to be loaded into memory, either in batches or entirely, depending on the training strategy. Large datasets can occupy a substantial portion of memory.<br/>

3- find out the actual Open Ai's training configuration of Clip model.<br/><br/>

OpenAI trained CLIP on a massive dataset of 400 million image-text pairs collected from the internet. They used a contrastive learning objective to train a dual-encoder architecture consisting of an image encoder and a text encoder. The model was trained on 592 V100 GPUs for 32 epochs. The batch size was 32,768. The image encoder was a ResNet-50, and the text encoder was a Transformer with 12 layers and 8 attention heads. More information can be found in the CLIP paper or [redacted link].


#### Bonus
4- We have an alternative clip's loss implementation, write its pseudocode.

In [ ]:
function calculate_alternative_clip_loss(image_embeddings, text_embeddings, temperature):
  # Normalize embeddings
  image_embeddings = normalize(image_embeddings)
  text_embeddings = normalize(text_embeddings)

  # Calculate similarity matrix
  similarity_matrix = image_embeddings @ text_embeddings.T / temperature

  # Calculate loss
  image_loss = cross_entropy_loss(similarity_matrix, targets)  # targets: identity matrix
  text_loss = cross_entropy_loss(similarity_matrix.T, targets)
  total_loss = (image_loss + text_loss) / 2

  return total_loss